In [ ]:
import os
import math
import pprint
import pyproj
import rasterio
import numpy as np
from pathlib import Path
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, mapping
from cutImg import openWaypointFile, clip_raster, representRaster
from preprocessCutPoint import lat_lon_to_epsg32654
from calculateCutPoint

In [140]:
def getPix4dRGBImgPath(filePath):
    RGBPathList = []
    DSMPathList = []

    for dateFileName in os.listdir(filePath):
        dateFileNamePath = filePath + '/' + dateFileName

        for step1 in os.listdir(dateFileNamePath):
            step1FilePath = dateFileNamePath + '/' + step1
            if Path(step1FilePath).is_dir():

                for step2 in os.listdir(step1FilePath):
                    if(step2 == "3_dsm_ortho"):
                        step2FilePath = step1FilePath + '/' + step2

                        for step3 in os.listdir(step2FilePath):
                            step3FilePath = step2FilePath + '/' + step3
                            if(step3 == "1_dsm"):
                                for step4 in os.listdir(step3FilePath):
                                    if(step4[-4:] == ".tif"):
                                        step4FilePath = step3FilePath + '/' + step4
                                        DSMPathList.append(step4FilePath)

                            if(step3 == "2_mosaic"):

                                for step4 in os.listdir(step3FilePath):
                                    if(step4[-4:] == ".tif"):
                                        step4FilePath = step3FilePath + '/' + step4
                                        RGBPathList.append(step4FilePath)
    return RGBPathList, DSMPathList

def openOffset(path):
    content = open(path,'r').read()
    content = content.split("\n")
    newDict = {}
    for line in content:
        line = line.split(' ')
        x,y = lat_lon_to_epsg32654(float(line[1]), float(line[2]))
        newDict[line[0]] = [float(line[1]), float(line[2])]
    return newDict
                

In [143]:
def calculateOffsetForCoor(coordinateList, coordinateOffsetDict):
    allCoorDict = {}
    for key, offsets in sorted(coordinateOffsetDict.items()):
        newCoordinateList = []
        for eachSet in coordinateList:
            newSetList = []
            for eachPoint in eachSet:
                x = eachPoint[0] + offsets[1]
                y = eachPoint[1] + offsets[0]
                newSetList.append([x,y])
            newCoordinateList.append(newSetList)
        allCoorDict[key] = newCoordinateList
    return allCoorDict

In [144]:
RGBFilePath = "D:/ice-wheat/data/Processed/MAVIC-RGB"
RGBFilePathList, DSMFilePathList = getPix4dRGBImgPath(RGBFilePath)
RGBFilePathList = RGBFilePathList[1:]
DSMFilePathList = DSMFilePathList[1:]
coordinateList = openWaypointFile("preprocessedCutPointFile.txt")
coordinateOffsetDict = openOffset("C:/Users/pacha/Desktop/masterProj/MasterProj/rawImage/offset.txt")
allCoorDict = calculateOffsetForCoor(coordinateList, coordinateOffsetDict)

In [154]:
def writeWayPointFile(content, fileName):
    # print(content)
    file = open(fileName, "a")
    file.write("<?xml version='1.0' encoding='utf-8' ?>")
    file.write("\n")
    file.write('<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:b="http://www.bizstation.jp/waypoint_extension" creator="Drogger GPS for Android 2.12.229" version="1.1">"')

    for x in content:
        for point in x:
            writeContent = '<wpt lat="' + str(point[1]) + '" lon="' + str(point[0]) + '">'
            file.write("\n")
            file.write(writeContent)
            file.write("\n")
            file.write("</wpt>")
            # <wpt lat="38.255167122" lon="140.828141869">
            # </wpt>

    file.write("\n")
    file.write("</gpx>")
    file.close()


In [ ]:
# writeWayPointFile(allCoorDict['202406171112'], '202406171112.gpx')
testRasterPath = RGBFilePathList[-2]
testCoor = allCoorDict['202406171112'][90]
outPath = './test90.tif'
clip_raster(testRasterPath,testCoor,outPath)